# Exam 1
## Joshua Anderson

### 1(a) Suppose we run a Monte Carlo simulation and draw n samples uniformly from \[ 0, 1 \]. Let $x_i$ be the ith sample. Write the formula for the estimate of the integral of f(x).

integral of f(x) = $\int_{0}^{0.5} 2-8x^{2}\ dx + \int_{0.5}^{1} 0\ dx = \frac{0.5}{n}$

#### 1(b)  For an estimate with 10 samples, on average, how many of the xi’s be below 0.5? What is the upperbound for the number of samples below 0.5? What is the lower bound?

Samples below 0.5 on average will be 5 for ten samples as the samples are drawn uniformially. The upper bound would be 10 and the lower bound would be 0 samples.

#### 1(c) Suppose we only draw 5 samples uniformly, and then use antithetic draws for the other 5. Give an example of what the 10 $x_i$’s could look like.

The antithetic sample would be $1-x_i$. So if we had 5 samples of \[0.2, 0.4, 0.5, 0, 1\], the antithetic samples would be \[0.8, 0.6, 0.5, 1, 0\].

The full sample would be \[0.2, 0.4, 0.5, 0, 1, 0.8, 0.6, 0.5, 1, 0\]

#### 1(d) Using the method in part c, on average, how many will be below 0.5? What is the upper bound for the number below 0.5? What is the lower bound?

Using the antithetic method, we should expect on average 5 samples below 0.5 if we are estimating 10 samples. The upper bound would be 5 and the lower bound would be 5.

#### 1(e) Would antithetic sampling reduce the variance of the estimate? Use Monte Carlo methods with both sampling methods and report on the variance of each method in your answer. (Hint: you will want to run multiple small sample Monte Carlo methods)


In [25]:
import random
import numpy as np

def f(x):
    if x <= 0.5:
        return 2 - (8 * x**2)
    else: 
        return 0

def area_random(samples, a, b, n):
    count = 0
    for _ in range(samples):
        x = random.uniform(0,1)
        count += f(x) > random.uniform(0,1)
        # else: count += 0 > random.uniform() would always be 0 so no code is needed
    I = (float(b-a)/n)*count
    return I

def area_antithetic(samples):
    count = 0
    for _ in range(samples//2):
        x = random.uniform(0,1)
        antithetic = 1-x
        if x <= 0.5:
            count += f(x) > random.uniform(0,1)
            count += f(antithetic) > random.uniform(0,1)
        # again, when x > 0.5 the area will always be 0; no else needed
    return count / samples

trials = 100000
samples = 10

reults_random = [area_random(samples) for _ in range(trials)]
print("Variance for random method:", round(np.var(reults_random), 4))
print("Variance for random method:", round(np.mean(reults_random), 4))
results_antithetic = [area_antithetic(samples) for _ in range(trials)]
print("Variance for antithetic method:", round(np.var(results_antithetic), 4))
print()
print("The antithetic method does reduce the variance in the estimate.")

TypeError: area_random() missing 3 required positional arguments: 'a', 'b', and 'n'

#### 1(f) Using the idea of importance sampling, draw samples from the uniform distribution between 0 and 0.5 such that $x ∼ U_0^{0.5}$. What is the formula for the integral of f(x) when you use this distribution?

Given:

$
f(x) = \left\{\begin{array}{ll}
      2-8x^{2} & x\leq 0.5 \\
      0 & 0.5 < x \\
\end{array}
\right.
$,
$p(x) = U_0^{0.5}$,
$q(x) = U_{0.5}^{1}$

The integral is constructed as:

$E[f(x)] = \int f(x)\frac{U_0^{0.5}}{U_{0.5}^{1}}$

#### 1(g). Is importance sampling better, worse, or the same compared to antithetic sampling? Why? Again, use Monte Carlo methods to compare importance sampling with antithetic sampling and the explain the results to answer the "why?" portion of the question.

In [19]:
from scipy import stats

# The target uniform distribution between 0 and 0.5 will have a mean of 0.25 and a variance of 1
# Our approximate distribution is the uniform distribution between 0.5 and 1 has a mean of 0.75 and variance of 1
target_mean = 0.25
target_var = 1
approx_mean = 0.75
approx_var = 1

p_x = stats.uniform(target_mean, target_var)
q_x = stats.uniform(approx_mean, approx_var)

trials = 1000

results_imp = []
for i in range(trials):
    x_i = np.random.normal(approx_mean, approx_var)
    value = f(x_i)*(p_x.pdf(x_i) / q_x.pdf(x_i))
    results_imp.append(value)

# we already have the variance from the antithetic sampling
print("Estimate for antithetic method:", round(np.mean(results_antithetic),2))
print("Variance for antithetic method:", round(np.var(results_antithetic), 4))
print("Estimate for importance method:", round(np.mean(results_imp),2))
print("Variance for importance method:", round(np.var(results_imp), 4))
print()
print("The importance sampling method is much worse. This is likely because the distributions do not overlap well. Using the distribution from 0.5 < x <= 1, we do not have a good representation of the distribution from 0 < x <= 0.5. The reason antithetic works much better is that for every sample it takes from the first interval [0,0.5] and samples from the second (0.5,1]. Since the distributions are so different on those intervals, the antithetic sampling gives a much better variance and estimate.")


Estimate for antithetic method: 0.22
Variance for antithetic method: 0.0123
Estimate for importance method: nan
Variance for importance method: nan
nan

The importance sampling method is much worse. This is likely because the distributions do not overlap well. Using the distribution from 0.5 < x <= 1, we do not have a good representation of the distribution from 0 < x <= 0.5. The reason antithetic works much better is that for every sample it takes from the first interval [0,0.5] and samples from the second (0.5,1]. Since the distributions are so different on those intervals, the antithetic sampling gives a much better variance and estimate.


In [2]:
!pip install ortools

In [14]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

wearhouse_costs = [22500, 38000]
wearhouse_limits = [240, 360]

ship_types = ["livestock","bio"]

ship_profits = {
    "livestock": 6,
    "bio": 8,
}

ship_startup = {
    "livestock": 22500,
    "bio": 38000,
}

ship_per_hour = {
    (0, "livestock"): 104,
    (0, "bio"): 76,
    (1, "livestock"): 84,
    (1, "bio"): 46
}

ship_locations = {(w, ship): solver.IntVar(0, wearhouse_limits[w], str(w) + "_" + ship) for w in range(len(wearhouse_costs)) for ship in ship_types}
ship_deltas = {(w, ship): solver.IntVar(0, 1, str(w) + "_" + ship) for w in range(len(wearhouse_costs)) for ship in ship_types}
wearhouse_deltas = [solver.IntVar(0, 1, str(w)) for w in range(len(wearhouse_costs))]

# l1 + l2 <= deltas

solver.Maximize(sum([ship_locations[(w, ship)] * ship_profits[ship] * ship_per_hour[(w,ship)] * ship_deltas[(w,ship)] for w in range(len(wearhouse_costs)) for ship in ship_types]) - 
                sum([ship_locations[w] * ship_startup[ship] for w in range(len(wearhouse_costs)) for ship in ship_types]))

# constraints
for w in range(len(wearhouse_costs)):
    solver.Add(ship_locations[(w, 'livestock')]*ship_startup['livestock'] + ship_locations[(w, 'bio')]*ship_startup['bio']  <= wearhouse_limits[w])
    
    for ship in ship_types:
        solver.Add(ship_locations[(w, ship)] <= wearhouse_deltas[w])

# solver.Add(sum([ for w in len(range(wearhouse_costs)) for ship in ship_types]))
  
# l1 240 l2 360 b1 0 b2 0

status = solver.Solve()

if status == solver.OPTIMAL:
    for k in ship_locations.keys():
        print(f'location: {k}, value: {ship_locations[k].solution_value()}')
    for w in range(len(wearhouse_costs)):
        for ship in ship_types:
            print((w,ship), wearhouse_deltas[w].solution_value())


TypeError: 

#### 2(a) Variables: 

$L_1$ - Number of livestock shipment hours in warehouse 1

$B_1$ - Number of biohazard shipment hours in warehouse 1

$L_2$ - Number of livestock shipment hours in warehouse 2

$B_2$ - Number of biohazard shipment hours in warehouse 2

#### 2(b) Objective function: 

$max((104*6)L_1 + (76*8)B_1 + (84*6)L_2 + (46*8)B_2 - (L_{cost1} + L_{cost2} + B_{cost1} + B_{cost2})$

#### 2(c) Constraints:

$L_1 + B_1 \leq 240$

$L_2 + B_2 \leq 360$

if $L_1$ >= 1, then subtract 22500
if $L_2$ >= 1, then subtract 22500
if $B_1$ >= 1, then subtract 38000
if $B_2$ >= 1, then subtract 38000

#### 3(a) You are bored on an international flight with a deck of cards as your only entertainment. How many different ways can you deal 5 cards from the deck (aka dealing a hand)? Note: It does not matter what order you receive the cards. The same set of cards in a different order is the same hand.

There are 13 different values of cards that can be of any suit:
$$\begin{pmatrix}13 \\ 5\end{pmatrix} = \dfrac{13!}{5!(13-5)!} = 1287$$
There are 1287 different hands.

#### 3(b) Later in the flight the passenger next to you suggests you play poker to pass the time. How many different ways can you deal two 5 card hands, one to each of you?